In [1]:
import logging

import pandas as pd
from simpletransformers.ner import NERModel, NERArgs
import re
import json
import wandb
import jsonlines

import os
import glob
import numpy as np
from sklearn.metrics import confusion_matrix
import numpy as np
import shutil

In [2]:
WANDB_NAME = "only_hearst_uniques"
EXPERIMENT_NAME = f"{WANDB_NAME}-split"

DATASET = '../data/musicians_dataset'
OUTPUTS = './outputs'
EXPERIMENTS = '../experiments'
LABELS = {"B": "B-MUS", "PB": "B-PER", "I": "I-MUS", "PI": "I-PER", "O": "O"}
DEV, TRAIN = "split_dev_only_hearst_uniques", "split_train_only_hearst_uniques"

In [3]:
BATCH_SIZE = 64
EPOCHS = 3
ARGS = {
        "seed": 42,
        "labels_list": list(LABELS.values()),
        "reprocess_input_data": True,
        "overwrite_output_dir": True,
        "train_batch_size": BATCH_SIZE,
        "eval_batch_size": BATCH_SIZE,
        "num_train_epochs": EPOCHS,
        "save_eval_checkpoints": False,
        "save_steps": -1,
        "use_multiprocessing": False,
        "use_multiprocessing_for_evaluation": False,
        "evaluate_during_training": True,
        "evaluate_during_training_steps": 50,
        "evaluate_during_training_verbose": True,
        "fp16": False,
        "wandb_project": WANDB_NAME,
        "learning_rate": 0.0003,
        "warmup_ratio": 0.1,
        "logging_steps": 1,
    }

In [4]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

### data preparation

In [5]:
def ingest_data_to_df(filepath, label_map):
    tagged_data = []
    sentence_number = 0
    with open(filepath, 'r') as f:
        for line in f.readlines():
            for tagged_word in line.split():
                try:
                    word, tag = tagged_word.split("-[", 1)
                    old_tag = tag.split("]", 1)[0]
                    new_tag = label_map[old_tag]
                    tagged_data.append([sentence_number, word, new_tag])
                except Exception as e:
                    raise Exception((tagged_word, line), e)
            sentence_number += 1
    return pd.DataFrame(tagged_data, columns=["sentence_id", "words", "labels"])

def get_dataset_parts(dataset_path, dev_name, train_name, test_name):
    devpath, trainpath, testpath = f"{dataset_path}/{dev_name}.txt", f"{dataset_path}/{train_name}.txt", f"{dataset_path}/{test_name}.txt"
    train = ingest_data_to_df(trainpath, LABELS)
    test = ingest_data_to_df(testpath, LABELS)
    dev = ingest_data_to_df(devpath, LABELS)
    return dev, train, test

def untag_dev_sentences(devpath):
    output_path = f"sentences_{devpath}"
    clean_sentences = []
    with open(devpath, "r") as fin:
        for line in fin.readlines():
            sentence = re.sub('\-\[[A-Z]*\]',  '', line)
            clean_sentences.append(sentence)
    return clean_sentences


# new data handling:
def import_jsons_to_df(dataset_path, filename):
    fp = f"{dataset_path}/{filename}.jsonl"
    tagged_data = []
    with jsonlines.open(fp, 'r') as f:
        for line in f:
            for token in line["sent_items"]:
                tagged_data.append([line["id"], token[0], token[1]])
    return pd.DataFrame(tagged_data, columns=["sentence_id", "words", "labels"])

def untagged_sentences(fp): 
    clean = []
    with jsonlines.open(fp, "r") as f:
        for line in f:
            words = [w[0] for w in line["sent_items"]]
            clean.append(" ".join(words))
    return clean

### train and evaluate with simpletransformers

In [6]:
def configure_model(args, model_type="roberta", model_name="roberta-base"):
    model = NERModel(
        model_type, 
        model_name, 
        args=args
    )
    return model

def send_prediction_to_conll(predictions, experiments_path):
    with open(f"{experiments_path}/predictions/{EXPERIMENT_NAME}.conll", "w") as f:
        for sent in predictions:
            for token_dict in sent:
                for k, v in token_dict.items():
                    f.write(f"{k} {v}\n")

def document_results(experiments_path, experiment_name, outputs):
    details = dict()
    with open(f"{experiments_path}/{experiment_name}.json", "w") as f:
        with open(f"{outputs}/best_model/model_args.json","r") as args:
            details["model_args"] = json.load(args)
        with open(f"{outputs}/best_model/config.json","r") as conf:
            details["best_model_conf"] = json.load(conf)
        with open(f"{outputs}/best_model/eval_results.txt","r") as res:
            details["eval_results"] = dict()
            for r in res.readlines():
                k, v = r.split (" = ")
                details["eval_results"][k] = v
        json.dump(details, f)

        
def main():
    dev_set = import_jsons_to_df(DATASET, DEV)
    train_set = import_jsons_to_df(DATASET, TRAIN)
    model = configure_model(args=ARGS)
    model.train_model(train_set, eval_data=dev_set)
    clean_sentences = untagged_sentences(f"{DATASET}/{DEV}.jsonl")
    predictions, raw_outputs = model.predict(clean_sentences)
    document_results(EXPERIMENTS, EXPERIMENT_NAME, OUTPUTS)
    send_prediction_to_conll(predictions, EXPERIMENTS)


In [7]:
main()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able

  0%|          | 0/18816 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

wandb: Currently logged in as: shovals (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Running Epoch 0 of 3:   0%|          | 0/294 [00:00<?, ?it/s]

...

  0%|          | 0/300 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.12265841141343117, 'precision': 0.7412060301507538, 'recall': 0.7763157894736842, 'f1_score': 0.7583547557840618}
INFO:simpletransformers.ner.ner_model: Training of roberta model complete. Saved to outputs/.
INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/300 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/5 [00:00<?, ?it/s]

## Error Analysis

In [ ]:
!ls ./outputs/best_model

In [8]:
%%bash -s "$EXPERIMENT_NAME" 
mv ./outputs/best_model ./best_models/$1

In [9]:
model = NERModel(
    "roberta", f"./best_models/{EXPERIMENT_NAME}"
)


In [10]:
gold_tags = []
pred_tags = []


with jsonlines.open(f"{DATASET}/{DEV}.jsonl", "r") as f:
    for line in f:
        words = [w[0] for w in line["sent_items"]]
        tokens = [w[1] for w in line["sent_items"]]
        predictions, raw_outputs = model.predict([" ".join(words)])
        if len(words) != len(predictions[0]):
            print(line)
        else:
            gold_tags.extend(tokens)
            pred_tags.extend([list(x.values())[0] for x in predictions[0]])

            
matrix = confusion_matrix(gold_tags, pred_tags, labels=['B-MUS', 'B-PER', 'I-MUS', 'I-PER', 'O'], normalize="true")

INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
print(np.round(matrix, 3))

[[0.296 0.643 0.    0.035 0.026]
 [0.015 0.863 0.    0.005 0.117]
 [0.    0.    0.321 0.649 0.03 ]
 [0.    0.006 0.019 0.911 0.065]
 [0.    0.007 0.    0.005 0.987]]


In [12]:
with open(f"./best_models/{EXPERIMENT_NAME}/eval_results.txt", "r") as f:
    print(f.read())

eval_loss = 0.11968257129192353
f1_score = 0.7756452680344142
precision = 0.7802929427430093
recall = 0.7710526315789473



In [ ]:
!ls ../data/musicians_dataset/

In [ ]:
%%bash -s "$TRAIN"
wc -l ../data/musicians_dataset/train_only_hearst.jsonl

In [ ]:
!wc -l ../data/lists/hearst_musicians.txt